# DSC670 - Week2 - Problem #4 - Extraction
## Using the OpenAI API, extract the text from this invoice(dsc-670-exercise-invoice.pdf file) and then use an LLM to convert the text to structured JSON (you can literally ask the LLM to produce JSON).


In [17]:
# Install required libraries. python-dotenv is required to load the env file which is required to save the sensitive information. Instead of hardcoding sensitive values (like your OpenAI API key) into your code, you store them in a .env file.
# !pip install openai python-dotenv
# PyPDF2 is a Python library used for working with PDF files. It allows you to read, manipulate, and write PDF documents without requiring external software like Adobe Acrobat.
#!pip install openai PyPDF2 

In [19]:
# Suppress all warnings
import warnings
warnings.filterwarnings("ignore")

In [35]:
# Step 1  Create an OpenAI client with API key

## Load required libraries
from openai import OpenAI
from PyPDF2 import PdfReader
import json
import os
from dotenv import load_dotenv

#Load variables from .env file into environment
load_dotenv()

#Create an OpenAI client with API key stored in env file
client = OpenAI(
    # Load the api key securely from env file.
    api_key=os.getenv("OPENAI_API_KEY") 
)

# --- Step 2: Load PDF ---
invoiceFileName = "dsc-670-exercise-invoice.pdf"

# Use PDF reader to extract the text from it
pdfText = ""
for page in reader.pages:
    pdfText += page.extract_text() + "\n"

#print(pdfText) # Verify content of pdf to text if all info is captured or not

# Step 2: Create a proper prompt to send it to LLM for json extraction

prompt = f"""Convert the following invoice text into structured JSON.
Make sure to include invoice metadata, company info, bill-to info, line items, taxes, discounts, and totals.
Invoice text:
{pdfText}
"""

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}],
    temperature=0
)

invoiceJson = response.choices[0].message.content

print("Structured JSON:")
print(invoiceJson)

Structured JSON:
Here is the structured JSON representation of the provided invoice text:

```json
{
  "invoice": {
    "metadata": {
      "invoice_number": "1111",
      "date": "2024-06-28",
      "po_number": "123456",
      "company_name": "Company Name",
      "contact": {
        "name": "<Name>",
        "phone": "<phone or email>"
      }
    },
    "company_info": {
      "address": "321 Avenue A",
      "city": "Portland",
      "state": "OR",
      "zip": "12345",
      "phone": "(206) 555-1163",
      "fax": "(206) 555-1164",
      "email": "someone@websitegoeshere.com"
    },
    "bill_to": {
      "name": "Natasha Jones",
      "company": "Central Beauty",
      "address": "123 Main St.",
      "city": "Manhattan",
      "state": "NY",
      "zip": "98765",
      "phone": "(321) 555-1234"
    },
    "line_items": [
      {
        "quantity": 1,
        "description": "Item Number 1",
        "unit_price": 2.00,
        "amount": 2.00
      },
      {
        "quantity":

## Analysis:
The JSON output structures invoice data well but lacks accuracy and proper business logic checks.

Placeholder fields like "<Name>" or "<phone or email>" show incomplete extraction
.
A negative balance_due, such as -994.20, suggests calculation errors; typically, credits lower totals, and a large negative indicates the vendor may owe the customer.

